# Problem Statement

Charging of electric arc furnace (EAF) can be made using two or three buckets. To operate using two buckets it is necessary to use high density scraps that are also more expensive. Charging can also be made with three buckets, if low density scraps (low cost) were used. The advantage of two bucket charging is the lower energy consumption (circa of 10 kWh/t) due to smaller heat losses during removal of furnace lid and to the shorter tap-to-tap time, since each bucket takes around 5 minutes to be discharged into furnace.

This exercise (originally an undergrad activity in Metallurgical Engineering course of UFMG, 2012) looks at the economical feasibility of each option (two or three bucket charging).

## Density and cost of different scrap types

In [ ]:
table1 = [
    {
        "scrap_class": "No. 1 Heavy",
        "type": "Heavy Weight scrap",
        "density": 0.85,
        "cost_per_ton": 320.
    },
    {
        "scrap_class": "No. 2 Heavy",
        "type": "Heavy Weight scrap",
        "density": 0.75,
        "cost_per_ton": 280.
    },
    {
        "scrap_class": "Internal Low Alloy",
        "type": "Heavy Weight scrap",
        "density": 3.,
        "cost_per_ton": 480.
    },
    {
        "scrap_class": "Plate & Structural",
        "type": "Heavy Weight scrap",
        "density": 2.,
        "cost_per_ton": 580.
    },
    {
        "scrap_class": "Internal Stainless Steel",
        "type": "Heavy Weight scrap",
        "density": 3.,
        "cost_per_ton": 660.
    },
    {
        "scrap_class": "No. 1 Bundles",
        "type": "Medium Weight scrap",
        "density": 1.2,
        "cost_per_ton": 360.
    },
    {
        "scrap_class": "No. 2 Bundles",
        "type": "Heavy Weight scrap",
        "density": 1.1,
        "cost_per_ton": 340.
    },
    {
        "scrap_class": "DRI",
        "type": "Heavy Weight scrap",
        "density": 1.65,
        "cost_per_ton": 440.
    },
    {
        "scrap_class": "No. 1 Busheling",
        "type": "Light scrap",
        "density": 1.50,
        "cost_per_ton": 420.
    },
    {
        "scrap_class": "Shredded",
        "type": "Light scrap",
        "density": 1.50,
        "cost_per_ton": 400.
    }
]

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(table1)
df

The goal is to produce a heat of 165 tons of liquid steel. However, due to alloy additions during the steel processing, charge must be 160t, maximum, using at most 3 scrap buckets. The volumetric capacity of EAF is 72 m³ (furnace dimensions: 3 m height, 5.5 diameter). But, after 1st bucket is completely melted, this volume is smaller due to volume occupied by liquid steel from previous chargings (liquid steel density = 7500 kg/m³). Other restrictions are:
 
-  Maximum amount of 30% (w/w) of heavy weight in each bucket to avoid electrode breakages
-  There is a need of a thin layer of light scrap on charge surface on scrap bucket to speed up electrode diving in the charge.

The goal is: 
1. Calculate the required weight in each bucket that results in the lowest cost for 3 buckets charging
2. Repeate the procedure for the scenario with 2 bucket charging.
3. Compare the cost of metallic charges between the two options (2 vs 3 buckets)

## Model

In [ ]:
# Solver instantiation
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver(
    name="two-buckets-problem", problem_type=pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

In [ ]:
# entities
buckets = {"first_bucket", "second_bucket"}
scrap_classes = set(map(lambda s: s["scrap_class"], table1))

# parameters
densities = {s["scrap_class"]: s["density"] for s in table1}
costs_per_ton = {s["scrap_class"]: s["cost_per_ton"] for s in table1}
bucket_volume = 72.

# decision variables
m_bs = {}
for bucket in buckets:
    m_bs[bucket] = {}
    for scrap_class in scrap_classes:
        scrap_class_wo_space = scrap_class.replace(' ', '')
        m_bs[bucket][scrap_class] = solver.NumVar(
            lb=0., ub=inf, name=f"y_{bucket}_{scrap_class_wo_space}")

print("Number of variables =", solver.NumVariables())

# objective value
obj_expr = []
for bucket in buckets:
    for scrap_class in scrap_classes:
        obj_expr.append(m_bs[bucket][scrap_class] * costs_per_ton[scrap_class])

solver.Minimize(solver.Sum(obj_expr))

In [ ]:
# run solver
print(solver.Solve())

In [ ]:
results = {}
for bucket in buckets:
    results[bucket] = {}
    for scrap_class in scrap_classes:
        results[bucket][scrap_class] = m_bs[bucket][scrap_class].SolutionValue()
        print(f"Bucket: {
              bucket} - Scrap class: {scrap_class} - Result: {results[bucket][scrap_class]}")

obj = solver.Objective().Value()
print(f"\n\nCharging cost: {obj}")